# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [13]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to visualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producteur:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
ventes:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

First, lets have a look to the first table, Wine :

In [7]:
%%sql
SELECT nv, cru, degre, mill
FROM Master1 
WHERE nv != "NONE" 
ORDER BY nv ASC

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973


We can conclude that in the table WINE, the attribute 'NV' is key => 3NF

Then, lets have a look to the second table, PRODUCTION :

In [8]:
%%sql
SELECT np, nv, qte 
FROM Master1 
ORDER BY np, nv

 * sqlite:///wine.db
Done.


NP,NV,QTE
None,9,None
None,40,90
None,54,210
None,63,136
None,66,124
None,89,980
1,1,300
1,3,400
1,54,500
2,45,90


We can conclude that in the table PRODUCTION , the attribute {'NP','NV'} is key => 3NF

Finally, lets have a look to the third table, PRODUCTEUR :

In [9]:
%sql SELECT np, nom, prenom, region FROM Master1 ORDER BY np ASC

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
None,None,None,None
None,None,None,None
None,None,None,None
None,None,None,None
None,None,None,None
None,None,None,None
1,Bohn,Rene,Alsace
1,Bohn,Rene,Alsace
1,Bohn,Rene,Alsace
2,Boxler,Albert,Alsace


We can conclude that in the table PRODUCTEUR, the attribute 'NP' is key => 3NF

Finally we have 3 tables :
1. WINE['NV', 'CRU', 'DEGRE', 'MILL'] with'NV' as a key
2. PRODUCTION ['NV', 'NP', 'QTE'] with'NV','NP' as a key
3. PRODUCTEUR['NP', 'NOM', 'PRENOM', 'REGION'] with'NP' as a key

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [10]:
First, lets have a look to the first table, WINE :

SyntaxError: invalid syntax (<ipython-input-10-0486dfa50116>, line 1)

In [ ]:
%%sql SELECT nv, cru, degre, mill 
FROM Master2 
WHERE nv != 'None'

We can conclude that in the table WINE, the attribute 'NV' is key => 3NF

Then, lets have a look to the first table, CUSTOMER :

In [ ]:
%%sql SELECT nb, nom, prenom, type 
FROM Master2 
WHERE nb != "None" 
ORDER BY nb
LIMIT 100

We can conclude that in the table CUSTOMER, the attribute 'NB' is key => 3NF

Then, lets have a look to the first table, VENTES :

In [ ]:
%%sql 
SELECT nv, nb, dates, lieu, qte 
FROM Master2
WHERE nb != 'None' and nv != 'None' and dates != 'None' and lieu != 'None'

We can conclude that in the table VENTES, the attribute {'NV','NB', 'DATE', 'LIEU'} is key => 3NF

Finally, lets have a look to the fourth table, LOCATION :

In [ ]:
%%sql 
SELECT lieu, region
FROM Master2
WHERE lieu != 'None'

We can conclude that in the table LOCATION, the attribute 'LIEU' is key => 3NF

Finally we have 4 tables :
1. WINE['NV', 'CRU', 'DEGRE', 'MILL'] with 'NV' as a key
2. CUSTOMERS['NB', 'NOM', 'PRENOM', 'TYPE'] with 'NB' as a key
3. VENTES['NV', 'NB', 'DATES', 'LIEU', 'QTE'] with {'NV','NB','DATES','LIEU'} as a key
4. LOCATION['LIEU', 'REGION'] with 'LIEU' as a key

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

In [ ]:
print("\nContent of the database")
printSchema(conn)

In [ ]:
%%sql
SELECT *
FROM dummy;

Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Notice that we use `NULL` given that `None` is only used for display.

In [ ]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

#### Exercise 1.3

Create the new tables from Master1:

In [14]:
%%sql 
DROP TABLE IF EXISTS wine;
DROP TABLE IF EXISTS production;
DROP TABLE IF EXISTS producteur;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER1
WHERE NV IS NOT NULL;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV,NP,QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;

-- Create producteur table
CREATE TABLE producteur AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [15]:
%%sql 
DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS ventes;
DROP TABLE IF EXISTS location;

-- Create customers table
CREATE TABLE customer AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

-- Create ventes table
CREATE TABLE ventes AS
SELECT DISTINCT NV,NB,DATES,LIEU,QTE
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL AND DATES IS NOT NULL AND LIEU IS NOT NULL;

-- Create location table
CREATE TABLE location AS
SELECT DISTINCT LIEU,REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [ ]:
print("\nContent of the database")
printSchema(conn)

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [16]:
%%sql
SELECT DISTINCT type
FROM customer

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [27]:
%%sql
SELECT DISTINCT region 
FROM wine
INNER JOIN production ON wine.NV = production.NV
INNER JOIN producteur p ON p.NP = production.NP
WHERE wine.CRU = 'Pommard' OR wine.CRU = 'Brouilly'

 * sqlite:///wine.db
Done.


REGION
Bourgogne
Rhone


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [29]:
%%sql
SELECT DISTINCT REGION
FROM wine w
INNER JOIN production pn ON w.NV = pn.NV
INNER JOIN producteur pr ON pr.NP = pn.NP
WHERE w.CRU = 'Pommard' AND w.CRU = 'Brouilly'

 * sqlite:///wine.db
Done.


REGION


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [55]:
%%sql
SELECT cru, mill, count(cru)
FROM wine w
INNER JOIN ventes v ON w.NV = v.NV
Group by cru, mill
ORDER BY cru

 * sqlite:///wine.db
Done.


CRU,MILL,count(cru)
Arbois,1980,1
Auxey Duresses,1914,1
Beaujolais Primeur,1983,2
Beaujolais Villages,1975,1
Beaujolais Villages,1976,1
Beaujolais Villages,1978,1
Beaujolais Villages,1979,3
Chapelle Chambertin,1973,3
Chateau Corton Grancey,1980,1
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [72]:
%%sql
SELECT w.NV, count(pr.NP) nbr_of_producer
FROM wine w
INNER JOIN production pn ON w.NV = pn.NV
INNER JOIN producteur pr ON pn.NP = pr.NP
GROUP BY w.NV
Having nbr_of_producer >=3


 * sqlite:///wine.db
Done.


NV,nbr_of_producer
12,3
43,3
45,5
54,3
67,3
69,3
74,3
78,5
79,3
87,3


#### Exercise 2.6

Which producers have not produced any wine?

In [90]:
%%sql
SELECT *
FROM producteur pr
WHERE pr.NP NOT IN (
    SELECT production.NP
    FROM production)

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [100]:
%%sql
SELECT DISTINCT c.prenom, c.nom, mill
FROM customer c
INNER JOIN ventes v ON c.NB = v.NB
INNER JOIN wine w ON v.NV = w.NV
WHERE mill = 1980


 * sqlite:///wine.db
Done.


PRENOM,NOM,MILL
Antonin,Artaud,1980
Andre,Gide,1980
Jean,Giono,1980
Louis,Aragon,1980
None,Lautreamont,1980
Stephane,Mallarme,1980


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [119]:
%%sql
SELECT c.prenom, c.nom, sum(CASE WHEN w.mill = 1980 THEN 1 ELSE 0 END) nbr80, mill
FROM customer c
INNER JOIN ventes v ON c.NB = v.NB
INNER JOIN wine w ON v.NV = w.NV
GROUP BY c.nom, c.prenom
HAVING nbr80 = 0

 * sqlite:///wine.db
Done.


PRENOM,NOM,nbr80,MILL
Emile,Ajar,0,1978
Yann,Andersen,0,1973
Jean,Anouilh,0,1978
Guillaume,Apollinaire,0,1968
None,Aristote,0,1975
Raymond,Aron,0,1978
Fernando,Arrabal,0,1975
Jacques,Audiberti,0,1974
Georges,Bataille,0,1975
Charles,Baudelaire,0,1943


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [127]:
%%sql
SELECT c.prenom, c.nom, sum(CASE WHEN w.mill = 1980 THEN 1 ELSE 0 END) nbr80, COUNT(c.nom) nbr_nom, COUNT(c.prenom) nbr_prenom, mill
FROM customer c
INNER JOIN ventes v ON c.NB = v.NB
INNER JOIN wine w ON v.NV = w.NV
GROUP BY c.nom, c.prenom
HAVING (nbr80 = nbr_nom OR nbr80 = nbr_prenom AND nbr80 != 0)

 * sqlite:///wine.db
Done.


PRENOM,NOM,nbr80,nbr_nom,nbr_prenom,MILL
Andre,Gide,18,18,18,1980
Jean,Giono,1,1,1,1980
None,Lautreamont,1,1,0,1980


#### Exercise 2.10

List all wines from 1980

In [139]:
%%sql
SELECT DISTINCT *
FROM wine
WHERE mill = 1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [132]:
%%sql
SELECT c.NB, cru, degre, mill
FROM wine w
INNER JOIN ventes v ON w.NV = v.NV
INNER JOIN customer c ON v.NB = c.NB
Where c.NB = 2

 * sqlite:///wine.db
Done.


NB,CRU,DEGRE,MILL
2,Mercurey,11.5,1980
2,Pommard,11.7,1976
2,Beaujolais Villages,11.8,1979


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [142]:
%%sql
SELECT c.NB, c.NOM, c.PRENOM, QTE, MILL
FROM customer c
INNER JOIN ventes v ON c.NB = v.NB
INNER JOIN wine w ON v.NV = w.NV
WHERE w.MILL =1980
GROUP BY c.NB
HAVING COUNT(*) = (SELECT COUNT(DISTINCT w.NV) FROM wine w WHERE w.MILL=1980)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,QTE,MILL
44,Gide,Andre,18,1980
